# Target Mask Generation & Visualization (Final)

This notebook verifies the **Garment-Independent Target Mask** logic using actual **SCHP model outputs**.

In [ ]:
import os, cv2, numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

def get_target_mask(parse_path, densepose_path):
    """
    Generates a neutral target mask using DensePose and Human Parsing.
    Handles sleeve variations and folded arms while preserving features.
    """
    parse = np.array(Image.open(parse_path).convert("L"))
    dp_segm = np.array(Image.open(densepose_path).convert("L"))

    # 1. Extract body surface from DensePose (Torso + Arms)
    guidance_mask = np.isin(dp_segm, [1, 2, 11, 12, 13, 14]).astype(np.uint8)

    # 2. Smooth and fill gaps (Occlusion handling)
    kernel = np.ones((5,5), np.uint8)
    neutral_target = cv2.morphologyEx(guidance_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    neutral_target = cv2.dilate(neutral_target, kernel, iterations=2)

    # 3. Preservation Check (Face, Hair, and now NECK 10)
    preserve_mask = np.isin(parse, [1, 2, 10, 13]).astype(np.uint8)
    neutral_target[preserve_mask == 1] = 0

    # 4. Construct Final Mask
    res_mask = parse.copy()
    res_mask[np.isin(parse, [5, 6, 7])] = 0 # Wipe original clothes
    res_mask[neutral_target == 1] = 5       # Set new neutral target as Label 5

    return res_mask

## Final Verification
Running visualization using the actual SCHP output from `outputs/schp`.

In [ ]:
# Project Paths
PROJECT_ROOT = Path("d:/Virtual try on/virtual-tryon")
DATA_ROOT = PROJECT_ROOT / "data/viton_hd/test"
if not DATA_ROOT.exists():
    DATA_ROOT = Path("/content/drive/MyDrive/virtual_tryon_project/data/viton_hd/test")

person_id = "00005_00"

img_path = DATA_ROOT / 'image' / f"{person_id}.jpg"
dp_path = DATA_ROOT / 'densepose' / f"{person_id}.png"

# PRIORITIZE YOUR GENERATED SCHP OUTPUT
parse_path = PROJECT_ROOT / 'outputs' / 'schp' / f"{person_id}.png"
if not parse_path.exists():
    parse_path = DATA_ROOT / 'image-parse-v3' / f"{person_id}.png"
    print("ℹ️ Note: Using dataset parse as fallback.")
else:
    print("✅ Using actual SCHP model output for verification.")

if all(p.exists() for p in [img_path, parse_path, dp_path]):
    target_mask = get_target_mask(parse_path, dp_path)
    
    fig, ax = plt.subplots(1, 4, figsize=(24, 12))
    ax[0].imshow(Image.open(img_path)); ax[0].set_title("1. Original Image")
    ax[1].imshow(Image.open(parse_path), cmap='nipy_spectral'); ax[1].set_title("2. SCHP Parsing Map")
    ax[2].imshow(Image.open(dp_path), cmap='viridis'); ax[2].set_title("3. DensePose Body Surface")
    ax[3].imshow(target_mask, cmap='nipy_spectral'); ax[3].set_title("4. AGNOSTIC TARGET MASK")
    
    for a in ax: a.axis('off')
    plt.tight_layout(); plt.show()
else:
    print("❌ Files missing. Check path structure.")